![unnamed (1).png](../MISC/unnamed(1).png)

# Analysing Customer Churn Data - Predicting Behavior to Retain Customers



Rayyan Minhaj (Data Science Intern - 10Pearls)

---

## Module 3: Database Schema, SQL, and Queries
Using SQL to create tables, perform data ingestion, call queries, and other various action items.

In [20]:
import psycopg2

In [23]:
conn = psycopg2.connect(host="localhost", dbname="10Pearls-Customer-Churn-Data", user="postgres", password="1234", port=5432)
cur = conn.cursor()

## 1. Creating Schema & Tables 

In [24]:
cur.execute("""
    CREATE TABLE Customers (
    customer_id VARCHAR(50) PRIMARY KEY,
    gender VARCHAR(10),
    senior_citizen BOOLEAN,
    partner BOOLEAN,
    dependents BOOLEAN
);

    CREATE TABLE Services (
    customer_id VARCHAR(50),
    phone_service BOOLEAN,
    multiple_lines BOOLEAN,
    internet_service VARCHAR(50),
    online_security BOOLEAN,
    online_backup BOOLEAN,
    device_protection BOOLEAN,
    tech_support BOOLEAN,
    streaming_tv BOOLEAN,
    streaming_movies BOOLEAN,
            
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

    CREATE TABLE Billing (
    customer_id VARCHAR(50),
    contract VARCHAR(50),
    paperless_billing BOOLEAN,
    payment_method VARCHAR(50),
    monthly_charges DECIMAL,
    total_charges DECIMAL,
            
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

            
    CREATE TABLE Churn_Predictions (
    customer_id VARCHAR(50),
    churn_prediction BOOLEAN,
    
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);        


""")

In [25]:
conn.commit()

![table_png.png](notebook_imgs/table_png.png)

### Services Table
![table_png.png](notebook_imgs/services_tbl.png)
### Customers Table
![table_png.png](notebook_imgs/customers_tbl.png)
### Churn Predictions Table
![table_png.png](notebook_imgs/churn_tbl.png)
### Billings Table
![table_png.png](notebook_imgs/billing_tbl.png)

## 2. Data Ingestion

In [27]:
import pandas as pd

raw_data = pd.read_csv('Raw_Data.csv')
telco_customer_churn = pd.read_csv('TelcoCustomerChurn.csv')
prediction_results = pd.read_csv('Prediction_Results.csv')

In [28]:
def insert_data(table_name, dataframe):
    for i, row in dataframe.iterrows():
        placeholders = ', '.join(['%s'] * len(row))
        columns = ', '.join(dataframe.columns)
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cur.execute(sql, tuple(row))
    conn.commit()

### Inserting into Customers Table

In [32]:
customers_data = telco_customer_churn[['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents']]
customers_data.columns = ['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents']
customers_data['senior_citizen'] = customers_data['senior_citizen'].astype(bool)  #Convert to boolean
customers_data['partner'] = customers_data['partner'].map({'Yes': True, 'No': False})  #Convert to boolean
customers_data['dependents'] = customers_data['dependents'].map({'Yes': True, 'No': False})  #Convert to boolean
insert_data('customers', customers_data)

C:\Users\Asus\AppData\Local\Temp\ipykernel_6484\3527777041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_data['senior_citizen'] = customers_data['senior_citizen'].astype(bool)  #Convert to boolean
C:\Users\Asus\AppData\Local\Temp\ipykernel_6484\3527777041.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_data['partner'] = customers_data['partner'].map({'Yes': True, 'No': False})  #Convert to boolean
C:\Users\Asus\AppData\Local\Temp\ipykernel_6484\3527777041.py:5: SettingWithCopyW

![table_png.png](notebook_imgs/customers_filled.png)

### Inserting into Services Table

In [33]:
services_data = telco_customer_churn[['customerID', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']]
services_data.columns = ['customer_id', 'phone_service', 'multiple_lines', 'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies']
services_data = services_data.applymap(lambda x: True if x == 'Yes' else False if x == 'No' else x)  #Convert 'Yes'/'No' to boolean
insert_data('services', services_data)

C:\Users\Asus\AppData\Local\Temp\ipykernel_6484\1937519431.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  services_data = services_data.applymap(lambda x: True if x == 'Yes' else False if x == 'No' else x)  # Convert 'Yes'/'No' to boolean


![table_png.png](notebook_imgs/services_filled.png)

### Inserting into Billings Table

In [34]:
billing_data = telco_customer_churn[['customerID', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']]
billing_data.columns = ['customer_id', 'contract', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges']
billing_data['paperless_billing'] = billing_data['paperless_billing'].map({'Yes': True, 'No': False})  # Convert to boolean
insert_data('billing', billing_data)

C:\Users\Asus\AppData\Local\Temp\ipykernel_6484\1764522316.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  billing_data['paperless_billing'] = billing_data['paperless_billing'].map({'Yes': True, 'No': False})  # Convert to boolean


![table.png](notebook_imgs/billing_filled.png)

### Inserting into Churn Predictions Table

In [45]:
# Prepare and insert data into Churn_Predictions table
prediction_data = prediction_results[['customerID', 'Churn']]
prediction_data.columns = ['customer_id', 'churn_prediction']

insert_data('churn_predictions', prediction_data)


![](notebook_imgs/churn_filled.png)

## 3. Querying the Data

In [42]:
cur.execute("ROLLBACK;")


In [10]:
conn.commit()

In [ ]:
cur.close()
conn.close()